# Add the defendant information to each charge

In [2]:
import pandas as pd
import numpy as np
import datetime
import csv

In [4]:
crim_defendant = pd.read_excel('../data/inputs/Defendant_12_04_2017.xlsx')
crim_defendant.head()

,Case Number,Unique Identifier,First Name,Last Name,DOB
0,32-2016-08713,3145473,AMANDA,PARRILLO,1985-05-31
1,31-2002-01546,4228143,KERRY,DION,1972-01-31
2,61-2002-00237,4236397,STEPHEN,SAPPO,1962-01-06
3,62-2010-12862,4240940,MELVIN,COLON,1974-11-20
4,61-2006-22994,4251314,SIRLEAF,TYLER,1978-02-12


In [5]:
charges_with_sentences = pd.read_excel('../data/steps/Step3Output_charges_sentences.xlsx')
charges_with_sentences.head()

,arresting_pd,case_id,case_no,charge_filing_date,charge_id,charge_number,disposition,disposition_date,judge,offense_date,offense_description,sentence,sentence_conditions,sentence_date,sentence_terms
0,MIDDLETOWN POLICE DEPARTMENT,7422794,21-2001-01905,2001-08-04,3909041,1,NaN,NaT,NaN,2001-08-03,Larceny < 500 Person 65+\n,NaN,NaN,NaT,
1,TIVERTON POLICE DEPARTMENT,7423292,21-2001-02507,2001-09-25,3909791,1,Plea of Nolo Contendere,2001-10-18,"Pirraglia (Retired), Robert K.",2001-09-24,Larceny < 500 Person 65+\n,Criminal Sentence,Probation 1Y Active: 10/18/2001; Court Cost...,2001-10-18,
2,Newport Police Department,7424561,21-2002-00802,2002-03-27,3911694,2,Dismissed 48A,2003-02-25,"Pirraglia (Retired), Robert K.",2002-03-27,FAILURE TO APPEAR FOR SUMMONS,NaN,NaN,NaT,
3,Newport Police Department,7424561,21-2002-00802,2002-03-27,3911693,1,Dismissed 48A,2003-02-25,"Pirraglia (Retired), Robert K.",2002-02-28,Larceny < 500 Person 65+\n,NaN,NaN,NaT,
4,Newport Police Department,7425369,21-2002-01767,2002-07-24,3913116,1,Plea of Nolo Contendere,2002-07-24,"Pirraglia (Retired), Robert K.",2002-07-11,Larceny < 500 Person 65+\n,Criminal Sentence,Credit for Time Served 10H Active: 07/24/20...,2002-07-24,


## Sub-step 1: Create a mapping from case number to defendant data

In [6]:
defendant_mapping = {}
for row in crim_defendant.iterrows():
    case_number = row[1]['Case Number']
    defendant_mapping[case_number] = row[1]

### Sub-step 2: Merge the charge data and defendant data

In [8]:
def snake_case(string):
    return string.lower().replace(' ', '_')

In [15]:
def create_merged_blob(charge, defendant):
    blob = {}
    for key in charge.keys():
        blob[snake_case(key)] = charge[key]
    for key in defendant.keys():
        if key == 'Case Number':
            continue
        blob[snake_case(key)] = defendant[key]
    return blob

In [16]:
merged_data = []
for row in charges_with_sentences.iterrows():
    case_no = row[1]['case_no']
    merged_blob = create_merged_blob(
        row[1],
        defendant_mapping[case_no]
    )
    merged_data.append(merged_blob)

In [17]:
to_write = pd.DataFrame(merged_data)
len(to_write)

3328

## Sub-step 3: Write data to `steps` folder

In [18]:
writer = pd.ExcelWriter('../data/steps/Step4Output_merged.xlsx')
to_write.to_excel(writer,'Sheet1')
writer.save()